In [1]:
import pandas as pd
import numpy as np

In [2]:
x_train_og = pd.read_csv('../data/processed/train_og.csv')

LABELS = {'Historia': 0, 'Administracao': 1, 'Geografia': 2, 'Biologia': 3, 
          'Literatura': 4, 'Artes': 5, 'Matematica': 6}


In [3]:
x_test = pd.read_csv('../data/processed/test.csv')

In [4]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("spacy", language='pt')
train_iter = iter([*x_train_og[['label', 'alt_title']].itertuples(index=False, name=None)])

def yield_tokens(data_iter):
    for _, rows in data_iter:
        yield tokenizer(rows)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["[UNK]"])
vocab.set_default_index(vocab["[UNK]"])

text_pipeline = lambda word: vocab(tokenizer(word))
label_pipeline = lambda idx: int(idx)

text_pipeline('Ola, tudo bem?')
# label_pipeline('1')

c:\Users\kevin\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchtext\data\utils.py:105: UserWarning: Spacy model "pt" could not be loaded, trying "pt_core_news_sm" instead
  warnings.warn(


[0, 0, 0, 0, 0]

In [5]:
train_iter = iter([*x_train_og[['label', 'alt_title']].itertuples(index=False, name=None)])

In [38]:
from torch.utils.data import DataLoader
import torch
from torchdata.datapipes.iter import IterableWrapper, ShardingFilter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64).to(device)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0).to(device)
    text_list = torch.cat(text_list).to(device)
    return label_list, text_list, offsets

train_iter = iter([*x_train_og[['label', 'alt_title']].itertuples(index=False, name=None)])
train_iter = IterableWrapper(train_iter)
# train_iter = ShardingFilter(train_iter)

dataloader = DataLoader(train_iter, shuffle=False, collate_fn=collate_batch)


In [37]:
processed_text = torch.tensor(text_pipeline('historia biologia administracao'), dtype=torch.int64)
offsets processed_text.size(0)

3

In [22]:
[*dataloader.dataset]

[(3, 'biologia organismos'),
 (6, 'matematica ensino medio projeto multiplo'),
 (3, 'biologia 3 genetica evolucao ecologia'),
 (0, 'tribos mal'),
 (0, 'egipcios'),
 (0, 'historia'),
 (0, 'caminhos fronteiras'),
 (0, 'grandes personagens historia universal'),
 (0, 'hitler desafia ordem mundial'),
 (6, 'aprender aplicar matematica'),
 (5, 'pantanal'),
 (6, 'fundamentos matematica elementar logaritimos'),
 (0, 'brasil ficou assim formacao fronteiras tratados limites'),
 (0, 'historia geral arte arquitetura xi'),
 (5, 'museu prado madrid'),
 (3, 'biologia programa completo'),
 (6, 'estudos matematica'),
 (0, 'questao agraria latifundio agricultura familiar'),
 (2, 'geografia conexoes estudos geografia geral brasil'),
 (0, 'pearl harbor chamas eua entram guerra'),
 (0, 'etiqueta antigo regime'),
 (2, 'nova geografia estudos geografia america'),
 (6, 'fundamentos matematica combinatoria probabilidade'),
 (0, 'memoria tempo igrejas sao paulo'),
 (0, 'seculo inacabado america desde 1900'),
 (6

In [23]:
[*dataloader]

[(tensor([3]), tensor([  5, 379]), tensor([0])),
 (tensor([6]), tensor([  3,  18,  41,  35, 977]), tensor([0])),
 (tensor([3]), tensor([  5,  22, 183,  65, 114]), tensor([0])),
 (tensor([0]), tensor([1210,  922]), tensor([0])),
 (tensor([0]), tensor([715]), tensor([0])),
 (tensor([0]), tensor([1]), tensor([0])),
 (tensor([0]), tensor([ 62, 117]), tensor([0])),
 (tensor([0]), tensor([ 327, 1044,    1,  443]), tensor([0])),
 (tensor([0]), tensor([831, 673, 378,  56]), tensor([0])),
 (tensor([6]), tensor([527, 522,   3]), tensor([0])),
 (tensor([5]), tensor([385]), tensor([0])),
 (tensor([6]), tensor([ 13,   3,  32, 907]), tensor([0])),
 (tensor([0]),
  tensor([   2,  779,  153,   66,  117, 1206,  346]),
  tensor([0])),
 (tensor([0]), tensor([   1,    7,    8,  101, 1251]), tensor([0])),
 (tensor([5]), tensor([ 17, 400, 350]), tensor([0])),
 (tensor([3]), tensor([  5, 205, 162]), tensor([0])),
 (tensor([6]), tensor([19,  3]), tensor([0])),
 (tensor([0]), tensor([ 95,  98, 342, 495, 769]),

In [7]:
from torch import nn


class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [8]:
train_iter = iter([*x_train_og[['label', 'alt_title']].itertuples(index=False, name=None)])
train_iter = ShardingFilter(train_iter)

num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [9]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()


def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [10]:
train_iter = iter([*x_train_og[['label', 'alt_title']].itertuples(index=False, name=None)])
test_iter = iter([*x_test[['label', 'alt_title']].itertuples(index=False, name=None)])

train_iter = ShardingFilter(train_iter)
test_iter = ShardingFilter(test_iter)

In [11]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

# Hyperparameters
EPOCHS = 10  # epoch
LR = 1  # learning rate
BATCH_SIZE = 16  # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

total_accu = None
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.80)
split_train_, split_valid_ = random_split(
    train_dataset, [num_train, len(train_dataset) - num_train]
)

train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)


In [12]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

-----------------------------------------------------------
| end of epoch   1 | time:  0.11s | valid accuracy    0.429 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  0.09s | valid accuracy    0.503 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time:  0.08s | valid accuracy    0.549 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time:  0.09s | valid accuracy    0.589 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  0.09s | valid accuracy    0.606 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time:  0.08s |

In [13]:
print("Checking the results of test dataset.")
accu_test = evaluate(test_dataloader)
print("test accuracy {:8.3f}".format(accu_test))

Checking the results of test dataset.
test accuracy    0.715


In [14]:
int2label = {v: k for k, v in LABELS.items()}
int2label

{0: 'Historia',
 1: 'Administracao',
 2: 'Geografia',
 3: 'Biologia',
 4: 'Literatura',
 5: 'Artes',
 6: 'Matematica'}

In [15]:
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item()
    
predict('biologia', text_pipeline)

3